Let $\tau_i$ denote the membrane time constant of neuron $i$. Then, up to the approximations related to the assumption of being in an Asynchronous Irregular state, we have the following forumalas for the mean and SD (standard deviation) of the input to different neurons. 

$\mu_i = \tau_i \sum_j W_{ij} r_j + h_i$

$\sigma^2_i = \tau_i \sum_j W_{ij}^2 r_j + \xi^2_i$

where $h_i$ and $\xi_i$ are the mean (which is positive) and SD of the external input, respectively.

And then the output firing rate of each neuron is given by 

$r_i = \Phi(\mu_i, \sigma_i) \qquad\qquad\qquad$     (1)

If I plug in the first two equations (where each is really $N$ equations) in the bottom one (again, really $N$ equations), then I obtain a system of $N$ equations for the $N$ unknowns, namely the $r_i$.  (part of) Your tasks is to solve this ssytem of equations. To do that, we will first promote these to time-dependent ODE's. These are auxiliary tools for us to find the fixed point, and are not really physical:

$T_i\frac{dr_i}{dt} + r_i = \Phi(\mu_i, \sigma_i)$

We will then solve these numerically using the Euler method (see my `utils_for_max.py`), and hope that the ODE's will converge onto a fixed point, which satisfies (1). The Euler discretization leads to

$r_i(t+1) = (1 - \alpha_i) r_i(t) + \alpha_i \Phi(\mu_i, \sigma_i)$

where I defined $\alpha_i = \frac{\delta t}{T_i}$. But to use `utils_for_max.Euler2fixedpt` you don't really need to do this, but you need to give it (as `dxdt`) the following:

$T_i^{-1} (-r_i + \Phi(\mu_i, \sigma_i))$ with the above expressions for $\mu$ and $\sigma$ plugged in. 


In the rest, I assume that you have already substituted the expressions for $\mu_i$ and $\sigma_i$, but won't write them explicitly for brevity.

In the 



In [1]:
import numpy as np
from utils_for_max import Phi, Euler2fixedpt
import utils_for_max

In [2]:
# Parameters chosen by us

N = 20

# Auxiliary time constants for excitatory and inhibitory
T_alpha = 0.5
T_E = 0.01
T_I = 0.01 * T_alpha

# Membrane time constants for excitatory and inhibitory
tau_alpha = 1
tau_E = 0.01
tau_I = 0.01 * tau_alpha

In [3]:
# [Will be optimised] network structure parameters

# Top = to E, left = from E
J = np.array([[1, -1],
              [1, -1]])
# Convention is that the right column is negative
# For optimisation we want to impose a positivity constraint

P = np.array([[1, 1],
              [1, 1]])

w = np.array([[1, 1],
              [1, 1]])


In [4]:
# Randomly generated network

# Cell types and random weights
N_E = np.int(0.8 * N)
N_I = N - N_E
cell_types = (np.concatenate([np.zeros(N_E, int), np.ones(N_I, int)]))

C = np.random.randint(2, size=(N, N))
np.fill_diagonal(C, 0)

# Auxiliary time constant vector for all cells
T = cell_types * T_I + (1-cell_types) * T_E
T_inv = np.reciprocal(T)

# Membrane time constant vector for all cells
tau = cell_types * tau_I + (1-cell_types) * tau_E

# Matrix of J coefficient for each weight in W
J_full = np.zeros((N, N))
for i in range(N):
    for j in range(N):
        J_full[i, j] = J[cell_types[i], cell_types[j]]

# Weight matrix and squared weight matrix
W = J_full * C
W2 = np.square(W)


In [5]:
def get_mu_sigma(W, W2, r, h, xi2):
    # Find net input mean and variance given inputs
    
    mu = tau * (W @ r + h)
    sigma2 = tau * (W2 @ r + xi2)
    
    return mu, np.sqrt(sigma2)


def dxdt_fun(r):
    # Function to feed into Euler solver
    
    return T_inv * (Phi(*get_mu_sigma(W, W2, r, h, xi2), tau) - r)
    

In [6]:
# Solve

r_init = np.ones(N)
h = np.ones(N) * 2000 #IS THIS TOO HIGH?
xi2 = np.ones(N)

R, did_converge = utils_for_max.Euler2fixedpt(dxdt_fun, r_init)

In [7]:
get_mu_sigma(W, W2, R, h, xi2)

(array([21.71889442, 22.98993726, 25.11503983, 23.69359518, 24.87181058,
        23.96151055, 24.23591349, 23.50267587, 23.20465851, 22.35619793,
        21.39503823, 23.78551214, 24.67857999, 24.51779355, 24.38113603,
        24.08109993, 24.46087773, 22.90140223, 22.58826833, 21.7898242 ]),
 array([1.97971371, 1.96951592, 2.51930901, 2.1407127 , 2.20948197,
        2.20240011, 2.33833093, 2.34588467, 2.33635222, 2.28817671,
        2.12790439, 2.23995317, 2.79490945, 2.39784728, 2.4977361 ,
        2.67591621, 2.54192383, 2.23882469, 1.90270162, 2.2387946 ]))

In [8]:
print(R)
print(R-Phi(*get_mu_sigma(W, W2, R, h, xi2), tau))


[42.80633526 51.14056433 64.87936755 55.83232497 63.01509034 57.56558631
 59.43327637 55.02271487 53.17735299 47.75164814 40.99959662 56.55962545
 62.77888482 61.20082046 60.551853   59.08275974 61.09390224 51.10025567
 48.42469478 43.95278522]
[-0.00037861 -0.00049772 -0.00073923 -0.00058569 -0.000663   -0.00059302
 -0.00062106 -0.00059193 -0.00053511 -0.00047491 -0.00038605 -0.00058144
 -0.00073521 -0.00066628 -0.00068965 -0.00067149 -0.00021359 -0.00015913
 -0.00013825 -0.00013528]


In [9]:
tau

array([0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
       0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01])

In [16]:
x = np.zeros(10)
for i in range(1000):
    x += np.random.binomial(1, 0.1*np.array(range(10)))
    
x / 1000

array([0.   , 0.099, 0.181, 0.301, 0.382, 0.52 , 0.591, 0.718, 0.803,
       0.904])